In [1]:
from dotenv import load_dotenv
from pathlib import Path
import os

In [2]:
# Get environment variables
dotenv_path = Path('./.env')
load_dotenv(dotenv_path=dotenv_path)

os.environ["NEO4J_URI"] = os.getenv('uri')
os.environ["NEO4J_USERNAME"] = os.getenv('user_name')
os.environ["NEO4J_PASSWORD"] = os.getenv('password')
os.environ["GROQ_API_KEY"] = os.getenv('GROQ_API_KEY')

#### Testing GraphCypherQAChain

In [ ]:
# ### Query Knowledge Graph
# from langchain.chains import GraphCypherQAChain
# from langchain.graphs import Neo4jGraph
# from langchain_groq import ChatGroq

# # Connect to graph
# graph = Neo4jGraph()

# # Create LLM instance
# llm = ChatGroq(temperature=0, groq_api_key=os.environ["GROQ_API_KEY"], model_name="llama-3.1-70b-versatile")

# graph_schema = f"Context of the graph's schema: {graph.schema}"
# personality_traits = graph.query("MATCH (n:Personality_Trait) return n")
# personality_traits = [title['n']['title'] for title in personality_traits]

In [ ]:
# prompt = f"""Your role is to recommend career paths for me. You have access to a Neo4j Graph Database. Here is the graph's schema: {graph_schema} and here the Personality_Trait titles that you can use: {personality_traits} and not that job_zone resembles the level of experience required for the job from 1 till 5 as integers. Use the graph to supplement your answer. Here are some details about myself: I love people and I am a good listener. I enjoy observation and analysis. what careers would you recommend?"""

# chain = GraphCypherQAChain.from_llm(graph=graph, llm=llm, verbose=True, validate_cypher=True)
# response = chain.invoke({"query": prompt})

In [ ]:
# print(response['result'])

### Testing Tool Creation

In [ ]:
# from langchain_core.prompts import ChatPromptTemplate
# from langchain_groq import ChatGroq
# from langchain_core.pydantic_v1 import BaseModel
# from graph_functions import connect_to_database

# class QueryKnowledgeGraph(BaseModel):
#     f'''The contains details about Occupations and their related demands. This is the schema of the graph as context: {label_and_prop, rel, rel_and_prop}. In your cypher query do not use any words that are not found in the schema. Use the graphs relations, labels, and properties to supplement your answer. Return an answer to the question, appraoch you used, the node_id that you referred to, and the query you wrote.'''

#     answer: str
#     approach_used: str
#     node_id: str
#     cypher_query: str

#     def query_neo4j(query):
#         result = graph.query(query)
#         return result


# chat = ChatGroq(temperature=1, groq_api_key=os.environ["GROQ_API_KEY"], model_name="mixtral-8x7b-32768")
# structured_llm = chat.with_structured_output(QueryKnowledgeGraph, include_raw=True)

# answer = structured_llm.invoke("I love people and I love to listen, what would you recommend as a career?")

In [ ]:
# print(answer['parsed'].answer)
# print(answer['parsed'].approach_used)
# print(answer['parsed'].node_id)
# print(answer['parsed'].cypher_query)

In [ ]:
# driver = connect_to_database(uri=os.getenv('uri'), username = os.getenv('user_name'), password = os.getenv('password'))
# query = f"MATCH (n:Occupation)-[]-(m:`Personality Trait` {{title: 'Artistic'}}) RETURN n"
# # with driver.session() as session:
# #     result = session.run(query)
# #     result = result.value()
# #     driver.close()
# result = graph.query(query)
# result

### Back to videos

In [ ]:
# from langchain_groq import ChatGroq
# from langchain.prompts import ChatPromptTemplate

# chat = ChatGroq(temperature=0, groq_api_key=os.environ["GROQ_API_KEY"], model_name="mixtral-8x7b-32768")

# prompt = f"The neo4j graph database contains details about Occupations and their related demands. This is the schema of the graph as context:\n{label_and_prop}\n{rel}\n{rel_and_prop}\nIn your cypher query do not use any words that are not found in the schema. Use the graphs relations, labels, and properties to supplement your answer. Return an answer to the question, appraoch you used, the node_id that you referred to, and the query you wrote."

# prompt_template = ChatPromptTemplate.from_template(prompt)

# response = chat.invoke("I love people and I love to listen, what would you recommend as a career?")

In [ ]:
# print(response.content)

In [ ]:
## Adding memory to a conversation

from langchain.chains import ConversationChain # Depricated
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import RunnableSequence
from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory

llm = ChatGroq(temperature=0, groq_api_key=os.environ["GROQ_API_KEY"], model_name="mixtral-8x7b-32768")

from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory
memory = ConversationSummaryMemory(llm=llm)

conversation = ConversationChain(
  llm=llm,
  memory=memory,
  verbose=True
)

# To manually load previous memory
# memory.save_context({"input": "Hi"}, {"output": "What's up"})
# memory.load_memory_variables({})
# conversation.predict(input="Limit your answers to one line. Give me a  love poem.")
# print(memory.buffer)

### Chains

In [ ]:
# ## A chain of prompts returning one output from one input
# from langchain.chains import SequentialChain, LLMChain

# llm = ChatGroq(temperature=0.9, groq_api_key=os.environ["GROQ_API_KEY"], model_name="llama-3.1-70b-versatile")

# # prompt template 1: translate to english
# first_prompt = ChatPromptTemplate.from_template("Translate the following review to english:\n\n{Review}")
# chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key="English_Review") # chain 1: input= Review and output= English_Review

# second_prompt = ChatPromptTemplate.from_template("Can you summarize the following review in 1 sentence:\n\n{English_Review}")
# chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key="summary") # chain 2: input= English_Review and output= summary

# # prompt template 3: translate to english
# third_prompt = ChatPromptTemplate.from_template("What language is the following review:\n\n{Review}")
# chain_three = LLMChain(llm=llm, prompt=third_prompt, output_key="language") # chain 3: input= Review and output= language

# # prompt template 4: follow up message
# fourth_prompt = ChatPromptTemplate.from_template("Write a follow up response to the following summary in the specified language:\n\nSummary: {summary}\n\nLanguage: {language}")
# chain_four = LLMChain(llm=llm, prompt=fourth_prompt, output_key="followup_message") # chain 4: input= summary, language and output= followup_message

# # overall_chain: input= Review 
# # and output= English_Review,summary, followup_message
# overall_chain = SequentialChain(
#     chains=[chain_one, chain_two, chain_three, chain_four],
#     input_variables=["Review"], 
#     output_variables=["English_Review", "summary","followup_message"], 
#     verbose=True)

In [ ]:
# output = overall_chain.invoke("Ana b7ebbak men kel albe ya kbir")

# print("review => ", output['Review'], '\n')
# print("english review => ", output['English_Review'], '\n')
# print("summary => ", output['summary'], '\n')
# print("followup message => ", output['followup_message'], '\n')

### Conversational Flow

In [3]:
from dotenv import load_dotenv
from pathlib import Path
import os

# Get environment variables
dotenv_path = Path('./.env')
load_dotenv(dotenv_path=dotenv_path)

os.environ["NEO4J_URI"] = os.getenv('uri')
os.environ["NEO4J_USERNAME"] = os.getenv('user_name')
os.environ["NEO4J_PASSWORD"] = os.getenv('password')
os.environ["GROQ_API_KEY"] = os.getenv('GROQ_API_KEY')

In [4]:
### Query Knowledge Graph
from langchain.chains import GraphCypherQAChain
from langchain.graphs import Neo4jGraph
from langchain_groq import ChatGroq

# Connect to graph
graph = Neo4jGraph()

# Create LLM instance
llm = ChatGroq(temperature=0, groq_api_key=os.environ["GROQ_API_KEY"], model_name="llama-3.1-70b-versatile")

# Prepare graph schema and context of Personality_Trait(s)
graph_schema = f"Context of the graph's schema: {graph.schema}"
personality_traits = graph.query("MATCH (n:Personality_Trait) return n")
personality_traits = [title['n']['title'] for title in personality_traits]

In [ ]:
# Prepare initial prompt
prompt = f"""Your role is to recommend career paths for me. You have access to a Neo4j Graph Database. Here is the graph's schema: {graph_schema} and here the Personality_Trait titles that you can use: {personality_traits} and not that job_zone resembles the level of experience required for the job from 1 till 5 as integers. Use the graph to supplement your answer. Here are some details about myself: I love people and I am a good listener. I enjoy observation and analysis. what careers would you recommend?"""

# chain that queries the graph
chain = GraphCypherQAChain.from_llm(graph=graph, llm=llm, verbose=True, validate_cypher=True)
response = chain.invoke({"query": prompt})

In [11]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationChain
from langchain_core.runnables.history import RunnableWithMessageHistory

# chain 1: Greet User and Explain what we will be doing
# LLMChain: replaced by |
system_template = "You are a career guide. You will ask peronality questions based on RIASC model then based on the answers you will suggest career paths suitable for my answers. First, greet the user and explain the flow of what we will do."

prompt_template = ChatPromptTemplate.from_messages([('system', system_template), ('user', '{text}')])

memory = ConversationSummaryMemory(llm=llm)

conversation_chain = ConversationChain(llm=llm, memory=memory, verbose=True)



chain = RunnableWithMessageHistory(prompt_template | llm, get_session_history=True)
result = conversation_chain.invoke("Hello")
print(result)


# chain 2: Ask questions


## Conditions
# chain 3a If user asks question => reply to user and then continue asking

# chain 3b If questions are done => query graph

# chain 4 Interpret answers and graph's query then return suggestions

# chian 5 Flow doesn't end here, user can ask more questions.



#############
## Chains

# LLMCheckerChain: This chain uses a second LLM call to verify its initial answer. Use this when you have an extra layer of validation on the initial LLM call.

# LLMRouterChain: This chain uses an LLM to route between potential options.
# MultiPromptChain: This chain routes input between multiple prompts. Use this when you have multiple potential prompts you could use to respond and want to route to just one.
# MultiRetrievalQAChain: This chain uses an LLM to route input questions to the appropriate retriever for question answering.

## Retreiver
#  Self Query

ValidationError: 1 validation error for RunnableWithMessageHistory
get_session_history
  True is not callable (type=type_error.callable; value=True)